In [1]:
#i "nuget:/Users/peet/Sources/csml/CsML/bin/Release"
// #i "nuget:C:\Users\peet.vermeulen\source\repos\csml\CsML\bin\Release"
#r "nuget:CsML,*-*"
using CsML;

Restore sources /Users/peet/Sources/csml/CsML/bin/Release Installed Packages CsML, 1.0.0

In [3]:
var inputPath = "/Users/peet/Sources/csml/Tests/Data/led7.csv";
// inputPath = @"C:\Users\peet.vermeulen\source\repos\csml\Tests\Data\led7.csv";
var data = CsML.Utility.Matrix.FromCSV(inputPath, null, loadFromRow: 1);

In [4]:
var dataLength = data.GetLength(0);
var features = new double[dataLength, 7];
var target = new double[dataLength];
for (int r=0; r < data.GetLength(0); r++)
{
    for (int c=0; c < 7; c++)
    {
        features[r, c] = data[r, c];
    }
    target[r] = data[r, 7];
}

In [6]:
var profiler = new CsML.Utility.Features.Profiler(features, target);
profiler.HasOutliers(features)

True

In [8]:
(features, target) = CsML.Utility.Features.Shuffle(features, target);
CsML.Utility.Features.ClassProportions<double>(target)

index,Item1,Item2,Item3
0,0,325,0.1015625
1,1,333,0.1040625
2,2,319,0.0996875
3,3,270,0.084375
4,4,336,0.105
5,5,335,0.1046875
6,6,341,0.1065625
7,7,304,0.095
8,8,327,0.1021875
9,9,310,0.096875


In [11]:
var results = new List<double>(){};
var iter = new CsML.Utility.KFoldIterator(dataLength, 10);
double[,] ftrain, ftest;
double[] ttrain, ttest;
foreach(bool[] f in iter)
{    

    Console.Write(".");
    (ftrain, ftest) = CsML.Utility.Matrix.Split(features, f);
    (ttrain, ttest) = CsML.Utility.Array.Split(target, f);
    var rcfier = new CsML.Probability.Classification.RandomClassifier<double>();
    rcfier.Train(ftrain, ttrain);
    double[] predictions = rcfier.Predict(ftest);
    results.Add(CsML.Utility.Array.ClassificationAccuracy(ttest, predictions));
}
var mean = results.Average();
results = results.Select(x => Math.Round(x, 4)).ToList();
Console.WriteLine("");
Console.WriteLine(string.Join(", ", results.ToArray()));
Console.WriteLine($"Average {mean}");

..........
0.1031, 0.0719, 0.1125, 0.075, 0.0906, 0.1125, 0.0844, 0.1188, 0.1125, 0.1
Average 0.098125


In [12]:
results = new List<double>(){};
iter = new CsML.Utility.KFoldIterator(dataLength, 10);
foreach(bool[] f in iter)
{
    Console.Write(".");
    (ftrain, ftest) = CsML.Utility.Matrix.Split(features, f);
    (ttrain, ttest) = CsML.Utility.Array.Split(target, f);
    var tree = new CsML.Tree.BinaryTree("classify", CsML.Utility.Statistics.Gini);
    tree.maxdepth = 15;
    tree.minrows = 3;
    tree.Train(ftrain, ttrain);
    double[] predictions = tree.Predict(ftest);
    results.Add(CsML.Utility.Array.ClassificationAccuracy(ttest, predictions));
}
var mean = results.Average();
results = results.Select(x => Math.Round(x, 4)).ToList();
Console.WriteLine("");
Console.WriteLine(string.Join(", ", results.ToArray()));
Console.WriteLine($"Average {mean}");

..........
0.7375, 0.7125, 0.7594, 0.6969, 0.7344, 0.6938, 0.7219, 0.75, 0.725, 0.7625
Average 0.7293749999999999


In [13]:
results = new List<double>(){};
iter = new CsML.Utility.KFoldIterator(dataLength, 10);
foreach(bool[] f in iter)
{
    Console.Write(".");
    (ftrain, ftest) = CsML.Utility.Matrix.Split(features, f);
    (ttrain, ttest) = CsML.Utility.Array.Split(target, f);
    var forest = new CsML.Tree.RandomForest("classify", CsML.Utility.Statistics.Gini);
    forest.treeCount = 500;
    forest.Train(ftrain, ttrain);
    double[] predictions = forest.Predict(ftest);
    results.Add(CsML.Utility.Array.ClassificationAccuracy(ttest, predictions));
}
var mean = results.Average();
results = results.Select(x => Math.Round(x, 4)).ToList();
Console.WriteLine("");
Console.WriteLine(string.Join(", ", results.ToArray()));
Console.WriteLine($"Average {mean}");

..........
0.7469, 0.6969, 0.7625, 0.7125, 0.7438, 0.7062, 0.7344, 0.75, 0.7344, 0.7844
Average 0.7371874999999999
